# Ограниченная машина Больцмана (Restricted Boltzmann machine)

## Введение

<b> Ограниченная машина Больцмана (RBM) </b> — это генеративная стохастическая искусственная нейронная сеть, которая определяет распределение вероятности на наборе входных данных.

Как следует из их названия, RBM являются вариантом машин Больцмана с ограничением, что их нейроны должны образовывать двудольный граф: пара узлов, сформированная из нейронов двух групп (обычно называемых «видимыми» и «скрытыми» соответственно), имеет симметричное соединение между собой, и нет никаких связей между узлами внутри группы. Это ограничение позволяет использовать более эффективные алгоритмы обучения, чем те, которые доступны для общего класса машин Больцмана, в частности, алгоритм контрастной дивергенции на основе градиента. Кроме того, особенностью ограниченных машин Больцмана является возможность прохождения обучение без учителя.

## Обозначения и вывод формул

Введем следующие обозначения:
<ul>
    <li>$N_v$ — количество нейронов видимого слоя </li>
    <li>$N_h$ — количество нейронов скрытого слоя </li>
    <li>$v_1, v_2, ..., v_{N_v}$ — состояния нейронов видимого слоя, $v_i \in \{0, 1\}, i = \overline{1, N_v}, v \in R^{N_v}$ </li>
    <li>$h_1, h_2, ..., h_{N_h}$ — состояния нейронов скрытого слоя, $h_i \in \{0, 1\}, i = \overline{1, N_h}, h \in R^{N_h}$ </li>
    <li>$W \in R^{N_v \times N_h}$ — матрица связей между нейронами видимого и скрытого слоя </li>
    <li>$w_{ij}$ — веc между $i$ -ым нейроном видимого слоя и $j$ - ым нейроном скрытого слоя ($i = \overline{1, N_v}, j = \overline{1, N_h}$)</li>
    <li>$a_i$ — смещение видимого нейрона ($i = \overline{1, N_v}$)</li>
    <li>$b_j$ — смещение скрытого нейрона ($j = \overline{1, N_h}$)</li>
</ul>

![RBM1](https://raw.githubusercontent.com/Frixinglife/RBM/adb5aa904ca892d3be8f05867935dc8075efc052/images/RBM1.svg)